# City scrapper

This notebook scraps Wikipedia route based on structured marked-up language patterns on the seek of city coordinates. There are plenty of applications, but the most promising is logistics.   

In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

host = 'https://en.wikipedia.org'
head_route = '/wiki'
tail_route = '/List_of_municipalities_of_Brazil'

In [2]:
fname='./cities_coords.csv'
fname_new='./cities_coords_new.csv'

df=pd.read_csv(fname)

df = df.replace(',','.', regex=True)

df=df[['URL','latitude', 'longitude']]

df['latitude']=pd.to_numeric(df['latitude'])
df['longitude']=pd.to_numeric(df['longitude'])

df['latitude']=df['latitude'].astype(str)
df['longitude']=df['longitude'].astype(str)

df['latitude']=df['latitude'].str.replace('.', ',')
df['longitude']=df['longitude'].str.replace('.', ',')

df.to_csv(fname)


/tmp/ipykernel_230269/3381613054.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['latitude']=df['latitude'].str.replace('.', ',')
/tmp/ipykernel_230269/3381613054.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['longitude']=df['longitude'].str.replace('.', ',')


In [3]:
head_route = '/wiki'
tail_route = '/List_of_municipalities_of_Brazil'
route = head_route + tail_route

uri = host+route

css_query = "#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"

r = req.get(uri)

soup = BeautifulSoup(r.text, 'html.parser')

print(soup)


<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-page-tools-enabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Municipalities of Brazil - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feat

In [4]:
import pandas as pd

fname='./cities_old.csv'

df_old=pd.read_csv(fname)

valid_columns=['URL', 'latitude', 'longitude']

has_underscore=lambda string: '_' in str(string)

condition_1=pd.isna(df_old['longitude']) & ~pd.isna(df_old['latitude'])
condition_2=df_old['latitude'].apply(has_underscore)
condition_3=pd.isna(df_old['latitude']) & pd.isna(df_old['longitude']) 

condition=condition_1 | condition_2 | condition_3

df_split=df_old[condition_1 | condition_2]
df_na=df_old[condition_3]

df_split['URL']=df_split['URL']+df_split['latitude']
df_split['latitude']=df_split['longitude']
df_split['longitude']=df_split['Unnamed: 3']

df_clean=df_old[~condition]

df_split=df_split[valid_columns]
df_na=df_na[valid_columns]
df_clean=df_clean[valid_columns]

condition=pd.isna(df_split['latitude']) & pd.isna(df_split['longitude'])

df_split_na=df_split[condition]
df_split=df_split[~condition]

df_na=pd.concat([df_split_na, df_na])

df_old=df_old[valid_columns]

/tmp/ipykernel_230269/2941496250.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_split['URL']=df_split['URL']+df_split['latitude']
/tmp/ipykernel_230269/2941496250.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_split['latitude']=df_split['longitude']
/tmp/ipykernel_230269/2941496250.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [5]:
df_clean['latitude']=pd.to_numeric(df_clean['latitude'])
df_clean['longitude']=pd.to_numeric(df_clean['longitude'])

latitude_mask=condition=df_clean['latitude']<-180
longitude_mask=condition=df_clean['longitude']<-180

latitude_label='latitude'
longitude_label='longitude'

df_clean.loc[latitude_mask, latitude_label] = df_clean[latitude_label] / 1000
df_clean.loc[longitude_mask, longitude_label] = df_clean[longitude_label] / 1000


In [6]:
rows_length = lambda dataframe: print(len(list(dataframe[dataframe.columns[0]])))

rows_length(df_split)
rows_length(df_na)

1045
358


In [7]:
from json import loads

def get_geo_coordinates(url):
    r = req.get(url)
    soup = BeautifulSoup(r.text)

    scripts = str(soup.find('script').text).strip()

    code_equalities = [
        code_line.split('=') for code_line in scripts.split(';')
    ]

    code_equalities = list(
        filter(lambda el: len(el) == 2, code_equalities)
    )

    lh_terms = list(
        set([ code_equality[0] for code_equality in code_equalities ])
    )

    lr_dict = {
        lh_term: [] for lh_term in lh_terms
    }

    for code_equality in code_equalities:
        #print(code_equality)
        lh_key = code_equality[0]

        lr_dict[lh_key].append(code_equality[1])
    
    if('RLCONF' not in lr_dict.keys()):
        return ()
    else:
        field_str = "wgCoordinates"

        confs = list(
            filter(
                lambda rh_term: field_str in rh_term, 
                lr_dict['RLCONF']
            )
        )
        
        if(len(confs)==0):
            return ()
        else:
            conf = loads(confs[0])
        
            return (conf[field_str]['lat'], conf[field_str]['lon'])

In [8]:
from tqdm import tqdm

coords=[]

urls=list(df_na['URL'])

for url in tqdm(urls):
    geocoord=get_geo_coordinates(url)
    
    if(geocoord!=()):
        coords.append((url, geocoord))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [04:47<00:00,  1.25it/s]


In [9]:
list_na_recoverred=list(
    map(
        lambda data: (data[0], data[1][0], data[1][1]), 
        filter(lambda tuple_: tuple_[1]!=(), coords)
    )
)

df_na_recovered=pd.DataFrame(list_na_recoverred)

df_na_recovered.columns=['URL', 'latitude', 'longitude']


In [10]:
for url, latitude, longitude in list_na_recoverred:
    url_row=df_na[df_na['URL']==url]
    
    url_row['longitude']=latitude
    url_row['latitude']=longitude

    df_na.update(url_row)


/tmp/ipykernel_230269/1191135480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_row['longitude']=latitude
/tmp/ipykernel_230269/1191135480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_row['latitude']=longitude


In [11]:
df_new=pd.concat([df_clean, df_split, df_na])

latitude_mask=df_new['latitude']<-180
longitude_mask=df_new['longitude']<-180

latitude_label='latitude'
longitude_label='longitude'

df_new.loc[latitude_mask, latitude_label] = df_new[latitude_label] / 1000
df_new.loc[longitude_mask, longitude_label] = df_new[longitude_label] / 1000

df_new['URL'] = df_new['URL'].astype(str)
df_new['longitude']=df_new['longitude'].astype(str)
df_new['latitude']=df_new['latitude'].astype(str)

df_new['longitude']=df_new['longitude'].str.replace('.', ',')
df_new['latitude']=df_new['latitude'].str.replace('.', ',')


/tmp/ipykernel_230269/932572242.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_new['longitude']=df_new['longitude'].str.replace('.', ',')
/tmp/ipykernel_230269/932572242.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_new['latitude']=df_new['latitude'].str.replace('.', ',')


In [12]:
fname='./cities_coords_recovered_.csv'
df_new.to_csv(fname)
